In [10]:
"""
Home Win dataset

create dataset with home_win from GCP to GCP
"""

'\nHome Win dataset\n\ncreate dataset with home_win from GCP to GCP\n'

In [11]:
# create table Home win dataset

# Import the important packages
import pandas as pd
import numpy as np
import datetime as dt
import getpass

# import schedule table
# Connexion GBQ
import pandas_gbq
from google.oauth2 import service_account




In [12]:

# Connexion GCP

def get_keys():
    '''
    This function will return the path to Google Keys for each user.
    '''
    if getpass.getuser()=='antoinetl':
        credential_keys = '/Users/antoinetl/Documents/nhl_prediction/nhl_prediction/account_keys/hockey-prediction-qc-9c75aa8a78f9.json'
    elif getpass.getuser()=='philippejacques':
        credential_keys = '/Users/philippejacques/Desktop/Projet/HockeyPrediction/Google Keys/rational-world-288611-dbe647d5aaf9.json'
    return credential_keys


# Credential keys : L'emplacement du fichier JSON 
#credential_keys = get_keys()
credential_keys = '/Users/antoinetl/Documents/nhl_prediction/nhl_prediction/account_keys/hockey-prediction-qc-9c75aa8a78f9.json'

# Connexion GBQ
credentials = service_account.Credentials.from_service_account_file(
    credential_keys,
    )
pandas_gbq.context.credentials = credentials


# documentation
# https://pandas-gbq.readthedocs.io/en/latest/intro.html



In [14]:
# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "hockey-prediction-qc"

sql = """select * from `hockey_prediction_qc.historical_games_detailed` limit 100"""

df_schedule_raw = pandas_gbq.read_gbq(sql, project_id=project_id)



Downloading: 100%|██████████| 100/100 [00:00<00:00, 191.58rows/s]


In [15]:
# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "hockey-prediction-qc"

sql = """
SELECT Date, 
        gameID, 
        gameDate,
        teams_home_team_id, teams_away_team_id,
        teams_home_score,
        teams_away_score
from `hockey_prediction_qc.historical_games_detailed`
where gameType = 'R' and status_abstractGameState = 'Final'
order by date
"""
df_schedule_raw = pandas_gbq.read_gbq(sql, project_id=project_id)

sql = """
SELECT * from `hockey_prediction_qc.teams_calendar` 
order by date
"""

df_cal_teams = pandas_gbq.read_gbq(sql, project_id = project_id)

Downloading: 100%|██████████| 5732/5732 [00:01<00:00, 2896.02rows/s]


In [16]:
df_schedule = df_schedule_raw

# date transformation
df_cal_teams['date'] = pd.to_datetime(df_cal_teams['date']).dt.tz_localize(None)
df_schedule['Date'] = pd.to_datetime(df_schedule['Date']).dt.tz_localize(None)
 
# Create date lag
df_schedule['date_lag'] = df_schedule['Date'] - dt.timedelta(days=1)
df_schedule.head()

# Get tonight's result
df_schedule = pd.merge(df_schedule, df_cal_teams[['wins_last_1', 
                                                  'loss_last_1', 
                                                  'ot_last_1', 
                                                  'date','team']], 
                       left_on=['Date', 
                                'teams_home_team_id'], 
                       right_on = ['date','team'], suffixes=('', '_home'))

df_schedule = pd.merge(df_schedule, df_cal_teams[['wins_last_1', 
                                                  'loss_last_1', 
                                                  'ot_last_1', 
                                                  'date','team']], 
                       left_on=['Date', 
                                'teams_away_team_id'], 
                       right_on = ['date','team'], suffixes=('', '_away'))

df_schedule = df_schedule.rename(columns={"wins_last_1": "Home_win", 
                                          "loss_last_1": "Home_loss", 
                                          "ot_last_1": 'Home_ot', 
                                          'ot_last_1_away': 'Away_ot'})

df_schedule = df_schedule.drop(['wins_last_1_away', 
                                'loss_last_1_away', 
                                'Date', 
                                'gameDate', 
                                'date_away', 
                                'team_away', 
                                'team'], axis=1)
#df = pd.merge(df_schedule,df_cal_teams[['Key_Column','Target_Column']],on='Key_Column', how='left')

df_schedule.head()

,gameID,teams_home_team_id,teams_away_team_id,teams_home_score,teams_away_score,date_lag,Home_win,Home_loss,Home_ot,date,Away_ot
0,2017020003,22,20,3,0,2017-10-03,1.0,0.0,0.0,2017-10-04,0.0
1,2017020001,52,10,2,7,2017-10-03,0.0,1.0,0.0,2017-10-04,0.0
2,2017020002,5,19,4,5,2017-10-03,0.0,0.0,1.0,2017-10-04,0.0
3,2017020004,28,4,3,5,2017-10-03,0.0,1.0,0.0,2017-10-04,0.0
4,2017020005,6,18,4,3,2017-10-04,1.0,0.0,0.0,2017-10-05,0.0


In [17]:
#df = df_schedule.merge(df_cal_teams, left_on=['date_lag', 'teams_home_team_id'], right_on=['date','team'], how = 'left', suffixes=('_left', '_right'))
df = pd.merge(df_schedule, df_cal_teams,  how='left', 
              left_on=['date_lag', 'teams_home_team_id'], 
              right_on = ['date','team'], suffixes=('', '_home'))

In [18]:
games_df = pd.merge(df, df_cal_teams, left_on=['date_lag', 'teams_away_team_id'], right_on = ['date','team'], suffixes=("", '_away'))

In [19]:
pd.set_option('display.max_columns', None)
games_df.describe(include = 'all')

,gameID,teams_home_team_id,teams_away_team_id,teams_home_score,teams_away_score,date_lag,Home_win,Home_loss,Home_ot,date,Away_ot,team,season,date_home,games_played,tonights_game_id,last_game_id,wins,losses,ot,points,ppg,shots_for_avg,shots_against_avg,shots_blocked_avg,goals_for_avg,goals_against_avg,hits_avg,pim_avg,powerPlayGoals_for_avg,powerPlayGoals_against_avg,powerPlayOpportunities_avg,takeaways_avg,giveaways_avg,wins_last_1,loss_last_1,ot_last_1,points_last_1,ppg_last_1,wins_last_10,loss_last_10,ot_last_10,points_last_10,ppg_last_10,team_away,season_away,date_away,games_played_away,tonights_game_id_away,last_game_id_away,wins_away,losses_away,ot_away,points_away,ppg_away,shots_for_avg_away,shots_against_avg_away,shots_blocked_avg_away,goals_for_avg_away,goals_against_avg_away,hits_avg_away,pim_avg_away,powerPlayGoals_for_avg_away,powerPlayGoals_against_avg_away,powerPlayOpportunities_avg_away,takeaways_avg_away,giveaways_avg_away,wins_last_1_away,loss_last_1_away,ot_last_1_away,points_last_1_away,ppg_last_1_away,wins_last_10_away,loss_last_10_away,ot_last_10_away,points_last_10_away,ppg_last_10_away
count,1.258000e+03,1258.000000,1258.000000,1258.000000,1258.000000,1258,1258.000000,1258.000000,1258.000000,1258,1258.000000,1253.000000,1253,1253,1253.000000,1.253000e+03,1239,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1258.000000,1258,1258,1258.000000,1.258000e+03,1241,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000
unique,NaN,NaN,NaN,NaN,NaN,177,NaN,NaN,NaN,177,NaN,NaN,1,176,NaN,NaN,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,177,NaN,NaN,950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,2018-01-24 00:00:00,NaN,NaN,NaN,2017-11-22 00:00:00,NaN,NaN,20172018,2018-01-24 00:00:00,NaN,NaN,2017020967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20172018,2018-01-24 00:00:00,NaN,NaN,2017020923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,15,NaN,NaN,NaN,15,NaN,NaN,1253,15,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1258,15,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,2017-10-04 00:00:00,NaN,NaN,NaN,2017-10-05 00:00:00,NaN,NaN,NaN,2017-10-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-10-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,2018-04-07 00:00:00,NaN,NaN,NaN,2018-04-08 00:00:00,NaN,NaN,NaN,2018-04-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-04-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2.017021e+09,18.906995,18.886328,3.124801,2.809221,NaN,0.564388,0.336248,0.099364,NaN,0.133545,18.920192,NaN,NaN,41.090184,2.017021e+09,NaN,20.559457,15.676776,4.853951,66.532322,1.619211,32.031736,31.999913,14.620939,2.

In [20]:
games_df.head()

,gameID,teams_home_team_id,teams_away_team_id,teams_home_score,teams_away_score,date_lag,Home_win,Home_loss,Home_ot,date,Away_ot,team,season,date_home,games_played,tonights_game_id,last_game_id,wins,losses,ot,points,ppg,shots_for_avg,shots_against_avg,shots_blocked_avg,goals_for_avg,goals_against_avg,hits_avg,pim_avg,powerPlayGoals_for_avg,powerPlayGoals_against_avg,powerPlayOpportunities_avg,takeaways_avg,giveaways_avg,wins_last_1,loss_last_1,ot_last_1,points_last_1,ppg_last_1,wins_last_10,loss_last_10,ot_last_10,points_last_10,ppg_last_10,team_away,season_away,date_away,games_played_away,tonights_game_id_away,last_game_id_away,wins_away,losses_away,ot_away,points_away,ppg_away,shots_for_avg_away,shots_against_avg_away,shots_blocked_avg_away,goals_for_avg_away,goals_against_avg_away,hits_avg_away,pim_avg_away,powerPlayGoals_for_avg_away,powerPlayGoals_against_avg_away,powerPlayOpportunities_avg_away,takeaways_avg_away,giveaways_avg_away,wins_last_1_away,loss_last_1_away,ot_last_1_away,points_last_1_away,ppg_last_1_away,wins_last_10_away,loss_last_10_away,ot_last_10_away,points_last_10_away,ppg_last_10_away
0,2017020010,16,5,10,1,2017-10-04,1.0,0.0,0.0,2017-10-05,0.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,20172018,2017-10-04,1,2017020002,None,0,0,1.0,1.0,1.0,33.0,34.0,10.0,4.0,5.0,27.0,8.0,1.0,1.0,5.0,4.0,13.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.1
1,2017020012,26,4,2,0,2017-10-04,1.0,0.0,0.0,2017-10-05,0.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,20172018,2017-10-04,1,2017020004,None,1,0,0.0,3.0,3.0,31.0,35.0,17.0,5.0,5.0,21.0,12.0,3.0,2.0,5.0,5.0,9.0,1.0,0.0,0.0,3.0,3.0,1.0,0.0,0.0,3.0,0.3
2,2017020023,12,30,5,4,2017-10-06,1.0,0.0,0.0,2017-10-07,1.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,20172018,2017-10-06,1,2017020009,None,0,1,0.0,0.0,0.0,39.0,31.0,10.0,2.0,2.0,19.0,10.0,0.0,2.0,4.0,3.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2017020018,13,14,5,4,2017-10-06,1.0,0.0,0.0,2017-10-07,0.0,13.0,20172018,2017-10-06,1.0,2.017020e+09,None,0.0,1.0,0.0,0.0,0.0,36.0,25.0,12.0,3.0,3.0,20.0,6.0,0.0,1.0,4.0,3.0,8.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,14,20172018,2017-10-06,1,2017020014,None,1,0,0.0,3.0,3.0,25.0,36.0,17.0,5.0,3.0,26.0,8.0,1.0,0.0,3.0,9.0,7.0,1.0,0.0,0.0,3.0,3.0,1.0,0.0,0.0,3.0,0.3
4,2017020029,24,4,2,3,2017-10-06,0.0,0.0,1.0,2017-10-07,0.0,24.0,20172018,2017-10-06,1.0,2.017020e+09,None,1.0,0.0,0.0,3.0,3.0,41.0,30.0,18.0,5.0,4.0,27.0,18.0,0.0,2.0,4.0,2.0,13.0,1.0,0.0,0.0,3.0,3.0,1.0,0.0,0.0,3.0,0.3,4,20172018,2017-10-06,2,2017020012,2017020004,1,1,0.0,3.0,1.5,33.0,31.0,15.0,2.5,2.5,20.0,11.0,1.5,1.0,5.0,4.5,9.5,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,0.3


In [21]:
games_df_prep = games_df[games_df['games_played'] > 15]
games_df_prep['ppg_diff'] = games_df_prep['ppg'] - games_df_prep['ppg_away']
games_df_prep['ppg_last_10_diff'] = games_df_prep['ppg_last_10'] - games_df_prep['ppg_last_10_away']

<ipython-input-21-61ad6f5be208>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_df_prep['ppg_diff'] = games_df_prep['ppg'] - games_df_prep['ppg_away']
<ipython-input-21-61ad6f5be208>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_df_prep['ppg_last_10_diff'] = games_df_prep['ppg_last_10'] - games_df_prep['ppg_last_10_away']


In [22]:
home_win_df = games_df_prep[[
    'gameID',
    'date',
    'teams_home_team_id',
    'teams_away_team_id',
    'Home_win',
    'ppg',
    'ppg_away',
    'ppg_diff',
    'ppg_last_10',
    'ppg_last_10_away',
    'ppg_last_10_diff'
]]

In [23]:
home_win_df.head()

,gameID,date,teams_home_team_id,teams_away_team_id,Home_win,ppg,ppg_away,ppg_diff,ppg_last_10,ppg_last_10_away,ppg_last_10_diff
215,2017020226,2017-11-07,5,53,1.0,1.6250,0.500000,1.125000,1.6,0.7,0.9
218,2017020232,2017-11-08,10,30,1.0,1.6875,1.307692,0.379808,1.2,1.3,-0.1
219,2017020233,2017-11-08,3,6,1.0,1.4375,1.615385,-0.177885,2.0,1.8,0.2
221,2017020237,2017-11-09,8,30,0.0,1.3750,1.214286,0.160714,1.8,1.2,0.6
224,2017020238,2017-11-09,19,53,1.0,2.3125,0.470588,1.841912,2.5,0.7,1.8


In [25]:
# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "hockey-prediction-qc"

# TODO: Set table_id to the full destination table ID (including the dataset ID).
table_id = 'hockey_prediction_qc.home_win_df'

pandas_gbq.to_gbq(home_win_df, table_id, project_id=project_id, if_exists='replace')

1it [00:07,  7.96s/it]
